In [2]:
from prometheus_api_client import PrometheusConnect

prom = PrometheusConnect(url='https://prom-gpu.orai.network/', disable_ssl=True)

prom.all_metrics()

/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[':node_memory_MemAvailable_bytes:sum',
 'ALERTS',
 'ALERTS_FOR_STATE',
 'DCGM_FI_DEV_FB_FREE',
 'DCGM_FI_DEV_FB_USED',
 'DCGM_FI_DEV_GPU_TEMP',
 'DCGM_FI_DEV_MEMORY_TEMP',
 'DCGM_FI_DEV_MEM_CLOCK',
 'DCGM_FI_DEV_NVLINK_BANDWIDTH_TOTAL',
 'DCGM_FI_DEV_PCIE_REPLAY_COUNTER',
 'DCGM_FI_DEV_POWER_USAGE',
 'DCGM_FI_DEV_SM_CLOCK',
 'DCGM_FI_DEV_TOTAL_ENERGY_CONSUMPTION',
 'DCGM_FI_DEV_VGPU_LICENSE_STATUS',
 'DCGM_FI_DEV_XID_ERRORS',
 'DCGM_FI_PROF_DRAM_ACTIVE',
 'DCGM_FI_PROF_GR_ENGINE_ACTIVE',
 'DCGM_FI_PROF_PIPE_TENSOR_ACTIVE',
 'aggregator_discovery_aggregation_count_total',
 'aggregator_unavailable_apiservice',
 'aggregator_unavailable_apiservice_total',
 'apiextensions_openapi_v2_regeneration_count',
 'apiextensions_openapi_v3_regeneration_count',
 'apiserver_admission_controller_admission_duration_seconds_bucket',
 'apiserver_admission_controller_admission_duration_seconds_count',
 'apiserver_admission_controller_admission_duration_seconds_sum',
 'apiserver_admission_step_admission_dur

In [29]:
import json
from datetime import datetime, timedelta
query = """avg_over_time(container_memory_usage_bytes{namespace="gpu-operator"}[1m])"""
avg_query = """avg(avg_over_time(container_cpu_allocation{container!=""}[1m])) by (pod)"""
end_time = datetime.now()
start_time = end_time - timedelta(minutes=60)
avg_step = end_time-start_time
ret = prom.custom_query_range(query=query, start_time=start_time, end_time=end_time, step=60)
# print(len(ret), ret[0])
# avg_ret = prom.custom_query_range(query=query,start_time=start_time, end_time=end_time, step=60)
# with open('text.json','w') as f:
#     json.dump(ret,f,indent=4)
# print(avg_ret)
ret

/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'metric': {'container': 'gc',
   'endpoint': 'https-metrics',
   'id': '/kubepods.slice/kubepods-burstable.slice/kubepods-burstable-poda3f3a1de_c446_4ae1_aecf_6a4b88ccdf5a.slice/cri-containerd-ff5a167dc1d54d6c88a585afa2c935a94bd3a190bd9b344e6012459f0a020925.scope',
   'image': 'registry.k8s.io/nfd/node-feature-discovery:v0.16.3',
   'instance': '210.211.99.160:10250',
   'job': 'kubelet',
   'metrics_path': '/metrics/cadvisor',
   'name': 'ff5a167dc1d54d6c88a585afa2c935a94bd3a190bd9b344e6012459f0a020925',
   'namespace': 'gpu-operator',
   'node': 'vt-gpu-staking-a100',
   'pod': 'gpu-operator-node-feature-discovery-gc-84c5f6b88f-kptkw',
   'service': 'kube-prometheus-stack-kubelet'},
  'values': [[1724059127, '12787712'],
   [1724059187, '12789760'],
   [1724059247, '12830720'],
   [1724059307, '13023232'],
   [1724059367, '13023232'],
   [1724059427, '13152256'],
   [1724059487, '13021184'],
   [1724059547, '12890112'],
   [1724059607, '12890112'],
   [1724059667, '12896256'],
   [

In [66]:
def calculate_sum(data):
    # Extract the list of values
    values_list = data.get('values', [])
    
    # Initialize a variable to hold the sum
    total_sum = 0.0
    
    # Iterate through the values list and sum the second items (the numeric values)
    for timestamp, value in values_list:
        total_sum += float(value)
    
    return total_sum
query = """sum(avg_over_time(container_memory_usage_bytes{container!="", container!="POD"}[10m])) by (pod, namespace)"""

result = prom.custom_query_range(query=query, start_time=start_time,end_time=end_time,step='10m')
for record in result:
    print(record['metric'])
    print(calculate_sum(record)/7)
# with open('text.json','w') as f:
#     json.dump(result,f,indent=4)


/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'namespace': 'gpu-operator', 'pod': 'gpu-feature-discovery-kksvj'}
167464277.3333333
{'namespace': 'gpu-operator', 'pod': 'gpu-operator-75df4b9c5-t8czr'}
43869720.38095237
{'namespace': 'gpu-operator', 'pod': 'gpu-operator-node-feature-discovery-gc-84c5f6b88f-kptkw'}
12974860.19047619
{'namespace': 'gpu-operator', 'pod': 'gpu-operator-node-feature-discovery-master-8659767c6b-trhmv'}
124141568.0
{'namespace': 'gpu-operator', 'pod': 'gpu-operator-node-feature-discovery-worker-mwv8n'}
27869866.666666664
{'namespace': 'gpu-operator', 'pod': 'nvidia-container-toolkit-daemonset-w25n4'}
42397696.0
{'namespace': 'gpu-operator', 'pod': 'nvidia-device-plugin-daemonset-lwnm7'}
27277019.42857143
{'namespace': 'gpu-operator', 'pod': 'nvidia-mig-manager-bpwts'}
19604431.238095243
{'namespace': 'gpu-operator', 'pod': 'nvidia-operator-validator-8flxs'}
1449984.0
{'namespace': 'ingress-nginx', 'pod': 'ingress-nginx-controller-6dfcb8658d-kx7f6'}
1547214067.8095238
{'namespace': 'jupyterhub', 'pod': 'al

In [54]:
sum_query = """avg(avg_over_time(container_memory_working_set_bytes{container!="", container_name!="POD", container!="POD"}[10m])) by (container_name, container, pod_name, pod, namespace, instance, pod)""" 

#'rke2-coredns-rke2-coredns-autoscaler-b49765765-9fhjd' 51631445.33333333
prom.custom_query_range(query=sum_query,  start_time=start_time,end_time=end_time,step='10m')

# with open('text2.json','w') as f:
#     json.dump(sum_result,f,indent=4)

/home/kh4ngptt/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prom-gpu.orai.network'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'metric': {'container': 'alpine-pod',
   'instance': '210.211.99.160:10250',
   'namespace': 'jupyterhub',
   'pod': 'alpine-pod'},
  'values': [[1724059127, '62619648'],
   [1724059727, '62619648'],
   [1724060327, '62619648'],
   [1724060927, '62619648'],
   [1724061527, '62619648'],
   [1724062127, '62619648'],
   [1724062727, '62619648']]},
 {'metric': {'container': 'autoscaler',
   'instance': '210.211.99.160:10250',
   'namespace': 'kube-system',
   'pod': 'rke2-coredns-rke2-coredns-autoscaler-b49765765-9fhjd'},
  'values': [[1724059127, '51631308.8'],
   [1724059727, '51550617.6'],
   [1724060327, '51367321.6'],
   [1724060927, '51543449.599999994'],
   [1724061527, '51528294.400000006'],
   [1724062127, '51528294.39999999'],
   [1724062727, '51415449.6']]},
 {'metric': {'container': 'calico-node',
   'instance': '210.211.99.160:10250',
   'namespace': 'kube-system',
   'pod': 'rke2-canal-9h7zv'},
  'values': [[1724059127, '197063270.39999998'],
   [1724059727, '196208640'],
 